In [ ]:
import cv2
import random
import os

import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, Flatten, Input, Lambda, Dropout, Conv2D, MaxPool2D
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.regularizers import l2, l1
import keras.backend as K

import warnings
warnings.simplefilter('ignore')

In [ ]:
def read_batch_imgs(current_person):
    imgs = []
    for j in range(1, 11):
        img = cv2.resize(cv2.imread('processed_images/s{}/{}.png'.format(current_person, j), cv2.IMREAD_GRAYSCALE),
                         (105, 105))
        imgs += [[img]]
    
    return imgs # Returns the original images without any normalization.

def generate_images():
    train_x = []
    for i in range(1, 41):
        imgs = read_batch_imgs(i)
        imgs = np.array(imgs).astype('float32') / 255
        train_x.append(imgs)
    return train_x
  
train_x = generate_images()

In [ ]:
pairs = []
labels = []

for n in range(40):
    for i in range(9):
        img1, img2 = train_x[n][i], train_x[n][i + 1]
        pairs += [[img1, img2]]
        inc = random.randrange(1, 40) 
        dn = (n + inc) % 40
        pairs += [[train_x[n][i], train_x[dn][i] ]]
        labels += [1, 0]

pairs = np.array(pairs)
labels = np.array(labels)

In [ ]:
print("Pairs shape : ", pairs.shape)
print("Labels shape : ", labels.shape)

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

# Bu fonksiyonu modelimizin üreteceği çıktının boyutunu belirlemek için kullanacağız.
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [ ]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0) ))

In [ ]:
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.3, y_true.dtype)))
  
# Modelimizin eğitim ve test verisi üzerinde nasıl çalıştığını görebilmek için.
def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.3
    return np.mean(pred == y_true)

In [ ]:
input_shape = (1, 105, 105)

def base_model_cnn(input_shape):
    inp = Input(shape=input_shape)
     
    x = Conv2D(64, (10, 10), activation='relu', data_format='channels_first')(inp)
    x = MaxPool2D()(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, (7, 7), activation='relu')(x)
    x = MaxPool2D()(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(256, (4, 4), activation='relu')(x)
    x = MaxPool2D()(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(512, (4, 4), activation='relu')(x)
    x = Flatten()(x)
    
    x = Dense(4096, activation='sigmoid')(x)
    
    return Model(inp, x)

In [ ]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

base = base_model_cnn(input_shape)

processed_a = base(input_a)
processed_b = base(input_b)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
model = Model([input_a, input_b], distance)

In [ ]:
model.compile(optimizer='adam', loss=contrastive_loss, metrics=[accuracy])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(pairs, labels, test_size=0.2, random_state=42)

In [ ]:
history = model.fit([train_x[:, 0], train_x[:, 1]], train_y, epochs=40, batch_size=64, validation_split=0.2)

In [ ]:
preds = model.predict([train_x[:, 0], train_x[:, 1]])
acc = compute_accuracy(train_y, preds)
print("Train accuracy : ", acc)

preds = model.predict([test_x[:, 0], test_x[:, 1]])
acc = compute_accuracy(test_y, preds)
print("Test accuracy : ", acc)

In [ ]:
model.save_weights('model_weights.h5')